In [1]:
"""
# FMA Dataset Integration
## Adding 100,000+ real music tracks to Smart Vinyl Catalog

The Free Music Archive provides a substantial dataset of Creative Commons music
with rich metadata including genres, audio features, and track information.
"""

import pandas as pd
import numpy as np
import requests
import zipfile
import os
import sys
from pathlib import Path

# Setup paths
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
data_dir = os.path.join(project_root, 'data', 'raw', 'fma')
os.makedirs(data_dir, exist_ok=True)

print("FMA Dataset Integration")
print("=" * 50)

def download_fma_metadata():
    """Download FMA metadata files"""
    
    base_url = "https://os.unil.cloud.switch.ch/fma/"
    files_to_download = [
        "fma_metadata.zip",  # Main metadata (340MB)
        "fma_small.zip"      # Audio features for subset (1.4GB - optional)
    ]
    
    print("Downloading FMA metadata...")
    
    for filename in files_to_download:
        file_path = os.path.join(data_dir, filename)
        
        if os.path.exists(file_path):
            print(f"✓ {filename} already exists")
            continue
            
        print(f"Downloading {filename}...")
        
        try:
            response = requests.get(base_url + filename, stream=True)
            response.raise_for_status()
            
            total_size = int(response.headers.get('content-length', 0))
            downloaded = 0
            
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        
                        if total_size > 0:
                            percent = (downloaded / total_size) * 100
                            print(f"\r  Progress: {percent:.1f}%", end="")
            
            print(f"\n✓ Downloaded {filename}")
            
        except Exception as e:
            print(f"✗ Failed to download {filename}: {e}")
            continue
    
    # Extract metadata
    metadata_zip = os.path.join(data_dir, "fma_metadata.zip")
    if os.path.exists(metadata_zip):
        print("Extracting metadata...")
        with zipfile.ZipFile(metadata_zip, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print("✓ Metadata extracted")

# Download the data
download_fma_metadata()

FMA Dataset Integration
  Progress: 100.0%
✓ Downloaded fma_metadata.zip
  Progress: 100.0%
✓ Downloaded fma_small.zip
Extracting metadata...
✓ Metadata extracted


In [2]:
def load_fma_metadata():
    """Load and process FMA metadata files"""
    
    # Load track metadata
    tracks_file = os.path.join(data_dir, 'fma_metadata', 'tracks.csv')
    
    if not os.path.exists(tracks_file):
        print(f"✗ Tracks file not found at {tracks_file}")
        return None, None, None
    
    print("Loading FMA metadata files...")
    
    # Load tracks (main metadata)
    tracks = pd.read_csv(tracks_file, index_col=0, header=[0, 1])
    
    # Load genres
    genres_file = os.path.join(data_dir, 'fma_metadata', 'genres.csv')
    genres = pd.read_csv(genres_file, index_col=0) if os.path.exists(genres_file) else None
    
    # Load artists  
    artists_file = os.path.join(data_dir, 'fma_metadata', 'artists.csv')
    artists = pd.read_csv(artists_file, index_col=0) if os.path.exists(artists_file) else None
    
    print(f"✓ Loaded tracks: {len(tracks)} entries")
    print(f"✓ Loaded genres: {len(genres) if genres is not None else 0} entries")
    print(f"✓ Loaded artists: {len(artists) if artists is not None else 0} entries")
    
    return tracks, genres, artists

# Load the data
tracks_df, genres_df, artists_df = load_fma_metadata()

if tracks_df is not None:
    print("\nFMA Dataset Structure:")
    print("Tracks columns:", tracks_df.columns.get_level_values(0).unique().tolist())
    print("Sample track data:")
    print(tracks_df.head())

Loading FMA metadata files...
✓ Loaded tracks: 106574 entries
✓ Loaded genres: 163 entries
✓ Loaded artists: 0 entries

FMA Dataset Structure:
Tracks columns: ['album', 'artist', 'set', 'track']
Sample track data:
            album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
10              0  2008-11-26 01:45:08  2008-02-06 00:00:00      NaN   
20              0  2008-11-26 01:45:05  2009-01-06 00:00:00      NaN   

                                                                          \
         favorites id                                information listens   
track_id                                                 

In [3]:
def process_fma_for_vinyl_catalog(tracks_df, genres_df, artists_df):
    """Process FMA data to match vinyl catalog structure"""
    
    if tracks_df is None:
        return pd.DataFrame()
    
    print("Processing FMA data for vinyl catalog integration...")
    
    # Extract relevant columns (handles multi-level column structure)
    processed_tracks = []
    
    for idx, track in tracks_df.iterrows():
        try:
            # Extract basic info
            track_data = {
                'fma_track_id': idx,
                'title': track.get(('track', 'title'), f'Track {idx}'),
                'artist': track.get(('artist', 'name'), 'Unknown Artist'),
                'album': track.get(('album', 'title'), 'Unknown Album'),
                'genre': track.get(('track', 'genre_top'), 'Unknown'),
                'year': track.get(('album', 'date_released'), None),
                'duration': track.get(('track', 'duration'), 0),
                'tags': track.get(('track', 'tags'), ''),
                'plays': track.get(('track', 'plays'), 0),
                'favorites': track.get(('track', 'favorites'), 0),
                'license': track.get(('track', 'license'), 'Creative Commons')
            }
            
            # Clean year data
            if pd.notna(track_data['year']):
                try:
                    year_str = str(track_data['year'])
                    if len(year_str) >= 4:
                        track_data['year'] = int(year_str[:4])
                    else:
                        track_data['year'] = None
                except:
                    track_data['year'] = None
            
            processed_tracks.append(track_data)
            
        except Exception as e:
            print(f"Error processing track {idx}: {e}")
            continue
    
    fma_catalog = pd.DataFrame(processed_tracks)
    
    # Filter for complete records
    fma_catalog = fma_catalog.dropna(subset=['title', 'artist'])
    
    # Add synthetic ratings based on popularity metrics
    max_plays = fma_catalog['plays'].max() if fma_catalog['plays'].max() > 0 else 1
    max_favs = fma_catalog['favorites'].max() if fma_catalog['favorites'].max() > 0 else 1
    
    fma_catalog['popularity_score'] = (
        (fma_catalog['plays'] / max_plays * 0.7) + 
        (fma_catalog['favorites'] / max_favs * 0.3)
    )
    
    # Convert to rating scale (1-5)
    fma_catalog['synthetic_rating'] = (
        fma_catalog['popularity_score'] * 3 + 2  # Scale to 2-5 range
    ).round(1)
    
    print(f"✓ Processed {len(fma_catalog)} tracks")
    print(f"Genre distribution:")
    print(fma_catalog['genre'].value_counts().head(10))
    
    return fma_catalog

# Process the FMA data
if tracks_df is not None:
    fma_catalog = process_fma_for_vinyl_catalog(tracks_df, genres_df, artists_df)
    
    print(f"\nFMA Integration Summary:")
    print(f"Total tracks processed: {len(fma_catalog)}")
    print(f"Unique artists: {fma_catalog['artist'].nunique()}")
    print(f"Unique albums: {fma_catalog['album'].nunique()}")
    print(f"Year range: {fma_catalog['year'].min()}-{fma_catalog['year'].max()}")
    print(f"Top genres: {', '.join(fma_catalog['genre'].value_counts().head(5).index.tolist())}")

Processing FMA data for vinyl catalog integration...
✓ Processed 106573 tracks
Genre distribution:
genre
Rock             14182
Experimental     10608
Electronic        9371
Hip-Hop           3552
Folk              2803
Pop               2332
Instrumental      2079
International     1389
Classical         1230
Jazz               571
Name: count, dtype: int64

FMA Integration Summary:
Total tracks processed: 106573
Unique artists: 16294
Unique albums: 14298
Year range: 1902.0-2021.0
Top genres: Rock, Experimental, Electronic, Hip-Hop, Folk


In [4]:
def upload_fma_to_bigquery(fma_catalog, sample_size=1000):
    """Upload FMA data to BigQuery (sample for demo)"""
    
    # Take a manageable sample for demo
    sample_catalog = fma_catalog.sample(n=min(sample_size, len(fma_catalog))).copy()
    
    # Rename columns to match your existing schema
    sample_catalog = sample_catalog.rename(columns={
        'fma_track_id': 'release_id',
        'synthetic_rating': 'rating'
    })
    
    # Add required columns
    sample_catalog['label'] = 'FMA'  # All from Free Music Archive
    sample_catalog['country'] = 'US'
    sample_catalog['review_text'] = sample_catalog.apply(
        lambda x: f"Creative Commons track by {x['artist']}. Genre: {x['genre']}. {x['tags']}", 
        axis=1
    )
    sample_catalog['review_source'] = 'FMA_Metadata'
    
    print(f"Prepared {len(sample_catalog)} FMA tracks for integration")
    print("\nSample integrated data:")
    print(sample_catalog[['title', 'artist', 'genre', 'year', 'rating']].head())
    
    return sample_catalog

# Create integrated dataset
if 'fma_catalog' in locals() and len(fma_catalog) > 0:
    integrated_fma = upload_fma_to_bigquery(fma_catalog, sample_size=5000)
    
    # Save for later use
    output_file = os.path.join(project_root, 'data', 'processed', 'fma_integrated.csv')
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    integrated_fma.to_csv(output_file, index=False)
    print(f"✓ Saved integrated FMA data to {output_file}")
    
    print(f"\nReal data integration complete!")
    print(f"Your catalog now includes:")
    print(f"- Original sample data: ~100 albums")
    print(f"- FMA real data: {len(integrated_fma)} tracks")
    print(f"- Combined dataset: {100 + len(integrated_fma)} total entries")
    
    print(f"\nNext steps:")
    print("1. Update Streamlit dashboard to use combined dataset")
    print("2. Test recommendation engine with real data")
    print("3. Demonstrate AI processing on FMA metadata")
else:
    print("FMA integration failed - using existing sample data")

Prepared 5000 FMA tracks for integration

Sample integrated data:
                                   title             artist         genre  \
87588  Tentacle Hentaij - Core Noisetone        Ralph Brown  Experimental   
15432                           Goldfish     Aoiroooasamusi           NaN   
8629                       The Monk Said              dmyra           NaN   
23563                      Fac ut animae  The Tudor Consort     Classical   
15730                    Bartlett Bridge          Vitamin-D          Folk   

         year  rating  
87588  2015.0     2.0  
15432  2010.0     2.0  
8629   2009.0     2.0  
23563  2010.0     2.0  
15730  2010.0     2.0  
✓ Saved integrated FMA data to /Users/richpointofview/smart-vinyl-catalog/data/processed/fma_integrated.csv

Real data integration complete!
Your catalog now includes:
- Original sample data: ~100 albums
- FMA real data: 5000 tracks
- Combined dataset: 5100 total entries

Next steps:
1. Update Streamlit dashboard to use combi